# Displaying charts
This notebook will add a elevation chart as a popup.

In [ ]:
from gpxplotter import create_folium_map, read_gpx_file, add_segment_to_map
import folium
line_options = {'weight': 8}

the_map = create_folium_map(tiles='openstreetmap')
for track in read_gpx_file('example3.gpx'):
    for i, segment in enumerate(track['segments']):
        add_segment_to_map(the_map, segment, color_by='hr-zone-float',
                           cmap='viridis', line_options=line_options)

# To display the map in a Jupyter notebook:
the_map

In [ ]:
# Create a chart:
import vincent
import json
import numpy as np

idx = np.argmax(segment['elevation'])

data = {
    'x': segment['Distance / km'],
    'y': segment['elevation'],
}

WIDTH = 400
HEIGHT = 200


line = vincent.Line(data, iter_idx="x", width=WIDTH, height=HEIGHT)
line.axis_titles(x='Distance / km', y='Elevation / m')
line.x_axis_properties(title_offset=2)
line.y_axis_properties(title_offset=-10)
line_json = line.to_json()
line_dict = json.loads(line_json)


popup = folium.Popup(max_width=WIDTH+50, show=True)
chart = folium.Vega(line_dict, width=WIDTH+50, height=HEIGHT+50)
chart.add_to(popup)

marker = folium.Marker(
    location=segment['latlon'][idx],
    popup=popup,
    icon=folium.Icon(icon='star'),
)
marker.add_to(the_map)
the_map

In [ ]:
# Create a chart using altair:
import altair as alt
import pandas as pd

In [ ]:
def smooth(signal, points):
    """Smooth the given signal using a rectangular window."""
    window = np.ones(points) / points
    return np.convolve(signal, window, mode='same')

data2 = pd.DataFrame(
    {
        'dist': segment['Distance / km'],
        'elevation': segment['elevation'],
        'heart': smooth(segment['hr'], 51),
    }
)

In [ ]:
area1 = alt.Chart(data2).mark_area(fillOpacity=0.4, strokeWidth=5, line=True).encode(
    x=alt.X('dist', title='Distance / km'),
    y=alt.Y('elevation', title='Elevation / m'),
)

line2 = alt.Chart(data2).mark_line(strokeWidth=5).encode(
    x=alt.X('dist', title='Distance / km'),
    y=alt.Y(
        'heart', #title='Heart rate / bpm', titleColor='#1b9e77'),
         axis=alt.Axis(title='Heart rate / bpm', titleColor='#1b9e77'),
    ),
    color=alt.value('#1b9e77'),
)
chart = alt.layer(
    area1,
    line2,
    width=WIDTH,
    height=HEIGHT,
).resolve_scale(y='independent')

chart.title = 'Elevation & heart rate profile'



the_map = create_folium_map(tiles='openstreetmap')
add_segment_to_map(the_map, segment, color_by='hr-zone-float',
                   cmap='viridis', line_options=line_options)


chart_dict = json.loads(chart.to_json())
popup = folium.Popup(max_width=WIDTH+100, show=True)
chart_vega = folium.features.VegaLite(chart_dict, width=WIDTH+100, height=HEIGHT+50)
chart_vega.add_to(popup)
marker = folium.Marker(
    location=segment['latlon'][idx],
    popup=popup,
    icon=folium.Icon(icon='star'),
)
marker.add_to(the_map)
the_map